### Carga dos Dados da Camada Landing para Bronze
Esse projeto será construido levando em consideração o modelo de arquitetura sugerido na seguinte documentação ->
[Medallion Schema](https://www.databricks.com/sites/default/files/inline-images/building-data-pipelines-with-delta-lake-120823.png?v=1702318922)

Nessa camada geralmente mantemos os dados brutos de carga, portanto, ela possui as seguintes caracteristicas:
- Dados brutos fiéis a origem
- Não são feitos tratamentos ou transformações
- Não é necessário inferir schema

In [0]:
%run ../../Lakehouse/functions/functions

#### Variáveis de Path - Landing para Bronze

##### Diesel e GNV

In [0]:
path_landing_diesel_gnv = '/databricks/landing/anp/diesel_gnv_mes_vigente.csv'
path_bronze_diesel_gnv = '/databricks/bronze/anp/diesel_gnv/'

##### Etanol e Gasolina

In [0]:
path_landing_etanol_gasol = '/databricks/landing/anp/etanol_gasolina_mes_vigente.csv'
path_bronze_etanol_gasol = '/databricks/bronze/anp/etanol_gasolina/'

##### GLP

In [0]:
path_landing_glp = '/databricks/landing/anp/glp_mes_vigente.csv'
path_bronze_glp = '/databricks/bronze/anp/glp/'

#### Leitura - Tabela Diesel GNV - Camada Landing-Bronze
- Definição de delimitador (;)
- Afirmação de header (True : Booleano)
- formato .csv

In [0]:
df_diesel_gnv = (
    spark.read.format('csv')\
        .option('delimiter',';')
        .option('header', True)
        .load(path_landing_diesel_gnv)
    )

In [0]:
df_etanol_gasolina = (
    spark.read.format('csv')\
        .option('delimiter',';')
        .option('header', True)
        .load(path_landing_etanol_gasol)
    )

In [0]:
df_glp = (
    spark.read.format('csv')\
        .option('delimiter',';')
        .option('header', True)
        .load(path_landing_glp)
    )

#### Necessário renomear as colunas
- Os headers originais possuem caracteres que não são aceitáveis pelo parquet/delta
- PS: o formato **.parquet** não aceita espaços e caracteres especiais

In [0]:
df_diesel_gnv_mapping = (
    df_diesel_gnv
      .select([pyspark.sql.functions.col(col).alias(col.replace('-', '').replace('  ', '_').replace(' ', '_')) for col in df_diesel_gnv.columns])
)

In [0]:
df_etanol_gasolina_mapping = (
    df_diesel_gnv
      .select([pyspark.sql.functions.col(col).alias(col.replace('-', '').replace('  ', '_').replace(' ', '_')) for col in df_etanol_gasolina.columns])
)

In [0]:
df_glp_mapping = (
    df_diesel_gnv
      .select([pyspark.sql.functions.col(col).alias(col.replace('-', '').replace('  ', '_').replace(' ', '_')) for col in df_glp.columns])
)

#### Dados - Bronze Output

##### Diesel GNV 

In [0]:
df_diesel_gnv_mapping.write.format('delta')\
    .mode('overwrite')\
    .option('overwriteSchema', True)\
    .save(path_bronze_diesel_gnv)

##### Gasolina e Etanol

In [0]:
df_etanol_gasolina_mapping.write.format('delta')\
    .mode('overwrite')\
    .option('overwriteSchema', True)\
    .save(path_bronze_etanol_gasol)

##### GLP

In [0]:
df_glp_mapping.write.format('delta')\
    .mode('overwrite')\
    .option('overwriteSchema', True)\
    .save(path_bronze_glp)